In [ ]:
import pandas as pd
import seaborn as sn
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import joblib
%matplotlib inline
sn.set_context("talk")

PARAMS

In [ ]:
HDM = [60,50,50]
NCRD = [70,50,60]


In [ ]:
EPOCHS = 20
CNN_DIM = HDM[0]
CNNS_DIM = HMD[1]
GRU_DIM = HDM[2]

In [ ]:
#Select a respective datasets and encoder
ENCODER = joblib.load("../data/NCRD/ncrd95-uniprot-ma_clf_enc.joblib")
OUTDIM = len(ENCODER.categories_[0]) #Dense layer number of units
TRAIN = "../data/NCRD/TrainNcrd95-ma-align.csv"
TEST = "../data/NCRD/TestNcrd95-ma-align.csv"
OUTPUTDIR = "../models/"

In [ ]:
ENCODER.categories_[0]

In [ ]:
with tf.device("cpu"):
    train = pd.read_csv(TRAIN)
    train = train[["Class","Sequence"]]
    test = pd.read_csv(TEST)
    test =  test[["Class","Sequence"]]
    sequences = pd.concat([train,test], axis = 0).Sequence.apply(lambda x: " ".join(x))
len(sequences)

In [ ]:
MAX_LEN = max(pd.concat([train[train.Class != "NonR"],test[test.Class != "NonR"]], axis = 0).Sequence.apply(lambda x: len(x)))
MAX_LEN

In [ ]:
OUTDIM

In [ ]:
ENCODER.categories_

In [ ]:
train["Sequence"] = train.Sequence.apply(lambda x: " ".join(x))
X_train = tf.convert_to_tensor(train.Sequence.to_list())
y_train = ENCODER.transform(train.Class.to_numpy().reshape(-1,1)).toarray()

test["Sequence"] = test.Sequence.apply(lambda x: " ".join(x))
X_test = tf.convert_to_tensor(test.Sequence.to_list())
y_test = ENCODER.transform(test.Class.to_numpy().reshape(-1,1)).toarray()

In [ ]:
cnn = tf.keras.models.Sequential()        
cnn.add(tf.keras.layers.TextVectorization(max_tokens = 21 + 2, output_mode = "int", standardize = None, output_sequence_length = MAX_LEN))
cnn.add(tf.keras.layers.Embedding(input_dim = 21 + 2, output_dim = CNN_DIM, input_length = MAX_LEN, mask_zero = True))
cnn.add(tf.keras.layers.Conv1D(filters = 512,   kernel_size = 5, activation = "relu"))
cnn.add(tf.keras.layers.MaxPooling1D())
cnn.add(tf.keras.layers.Conv1D(filters = 512/2, kernel_size = 5, activation = "relu"))
cnn.add(tf.keras.layers.MaxPooling1D())
cnn.add(tf.keras.layers.Conv1D(filters = 512/4, kernel_size = 5, activation = "relu"))
cnn.add(tf.keras.layers.MaxPooling1D())
cnn.add(tf.keras.layers.Conv1D(filters = 512/8, kernel_size = 5, activation = "relu"))
cnn.add(tf.keras.layers.Flatten())
cnn.add(tf.keras.layers.Dense(OUTDIM, activation = 'softmax'))
cnn.layers[0].adapt(sequences)
cnn.compile(optimizer = tf.optimizers.Adam(learning_rate = 0.001),loss = tf.keras.losses.CategoricalCrossentropy(name = "CategoricalAccuracy"),metrics = [tf.metrics.CategoricalAccuracy(name = "Accuracy")])
cnn_h = cnn.fit(X_train, y_train, epochs=EPOCHS, validation_split=0.2)

In [ ]:
cnn_same = tf.keras.models.Sequential()        
cnn_same.add(tf.keras.layers.TextVectorization(max_tokens = 21 + 2, output_mode = "int", standardize = None, output_sequence_length = MAX_LEN))
cnn_same.add(tf.keras.layers.Embedding(input_dim = 21 + 2, output_dim = CNNS_DIM, input_length = MAX_LEN, mask_zero = True))
cnn_same.add(tf.keras.layers.Conv1D(filters = 512,   kernel_size = 5, activation = "relu"))
cnn_same.add(tf.keras.layers.MaxPooling1D())
cnn_same.add(tf.keras.layers.Conv1D(filters = 512, kernel_size = 5, activation = "relu"))
cnn_same.add(tf.keras.layers.MaxPooling1D())
cnn_same.add(tf.keras.layers.Conv1D(filters = 512, kernel_size = 5, activation = "relu"))
cnn_same.add(tf.keras.layers.MaxPooling1D())
cnn_same.add(tf.keras.layers.Conv1D(filters = 512, kernel_size = 5, activation = "relu"))
cnn_same.add(tf.keras.layers.Flatten())
cnn_same.add(tf.keras.layers.Dense(OUTDIM, activation = 'softmax'))
cnn_same.layers[0].adapt(sequences)
cnn_same.compile(optimizer = tf.optimizers.Adam(learning_rate = 0.001),loss = tf.keras.losses.CategoricalCrossentropy(name = "CategoricalAccuracy"),metrics = [tf.metrics.CategoricalAccuracy(name = "Accuracy")])
cnn_same_h = cnn_same.fit(X_train, y_train, epochs=EPOCHS, validation_split=0.2)

In [ ]:
gru = tf.keras.models.Sequential()        
gru.add(tf.keras.layers.TextVectorization(max_tokens = 21 + 2, output_mode = "int", standardize = None, output_sequence_length = MAX_LEN))
gru.add(tf.keras.layers.Embedding(input_dim = 21 + 2, output_dim = GRU_DIM, input_length = MAX_LEN, mask_zero = True))
gru.add(tf.keras.layers.GRU(512,return_sequences=True))
gru.add(tf.keras.layers.GRU(512,return_sequences=True))
gru.add(tf.keras.layers.GlobalAveragePooling1D())
gru.add(tf.keras.layers.Dense(OUTDIM, activation = 'sigmoid'))
gru.layers[0].adapt(sequences)
gru.compile(optimizer = tf.optimizers.Adam(learning_rate = 0.001),loss = tf.keras.losses.BinaryCrossentropy(name = "BinaryCrossentropy"),metrics = [tf.metrics.BinaryAccuracy(name = "BinaryAccuracy")])
gru_hist = gru.fit(X_train, y_train, epochs=EPOCHS, validation_split=0.2)

In [ ]:
sn.lineplot(x = list(range(1,EPOCHS+1)), y = gru_hist.history["loss"], label = "GRU-Training", c="blue")
sn.lineplot(x = list(range(1,EPOCHS+1)), y = gru_hist.history["val_loss"], label = "GRU-Validation", c="blue", linestyle='dashed')

sn.lineplot(x = list(range(1,EPOCHS+1)), y = cnn_same_h.history["loss"], label = "CNNS-Training", c="red")
sn.lineplot(x = list(range(1,EPOCHS+1)), y = cnn_same_h.history["val_loss"], label = "CNNS-Validation", c="red", linestyle='dashed')

sn.lineplot(x = list(range(1,EPOCHS+1)), y = cnn_h.history["loss"], label = "CNN-Training", c = "green")
sn.lineplot(x = list(range(1,EPOCHS+1)), y = cnn_h.history["val_loss"], label = "CNN-Validation",c = "green", linestyle='dashed')


plt.title("Model Convergence")
plt.xlabel("Epoch")
plt.ylabel("CategoricalCrossentropy")
plt.savefig(f"{OUTPUTDIR}/all-uniprot-align.svg", format="SVG", bbox_inches='tight')
plt.savefig(f"{OUTPUTDIR}/all-uniprot-align.png", format="PNG", bbox_inches='tight')

In [ ]:
print(OUTPUTDIR)

In [ ]:
cnn.summary()

In [ ]:
cnn.save(f"{OUTPUTDIR}/cnn")

In [ ]:
cnn_same.summary()

In [ ]:
cnn_same.save(f"{OUTPUTDIR}/cnn-same")

In [ ]:
gru.summary()

In [ ]:
gru.save(f"{OUTPUTDIR}/gru")